# Getting Familiar with FarmBurg

Brian is a Product Manager at FarmBurg, a company that makes a farming simulation social network game. In the FarmBurg game, you can plow, plant, and harvest different crops.

Today, you will be acting as Brian's data analyst for an A/B Test that he has been conducting.

Brian tells you that he ran an A/B test with three different groups: A, B, and C. You are kind of busy today, so you do not ask too many questions about the differences between A, B, and C. Maybe they were shown three different versions of an ad. Who cares?

(HINT: you will care later)

Brian gives you a CSV of results called **clicks.csv**. It has the following columns:

* `user_id`: a unique id for each visitor to the FarmBurg site
* `ab_test_group`: either A, B, or C depending on which group the visitor was assigned to
* `click_day`: only filled in if the user clicked on a link to purchase

***

1. Start by importing `pandas` as `pd`. You will be using this module for most of this project.

In [1]:
import pandas as pd

2. Load `clicks.csv` into the variable `df`.

In [2]:
df = pd.read_csv('clicks.csv')

3. Examine the first few rows of df using `.head()`

In [3]:
df.head()

,user_id,group,click_day
0,8e27bf9a-5b6e-41ed-801a-a59979c0ca98,A,NaN
1,eb89e6f0-e682-4f79-99b1-161cc1c096f1,A,NaN
2,7119106a-7a95-417b-8c4c-092c12ee5ef7,A,NaN
3,e53781ff-ff7a-4fcd-af1a-adba02b2b954,A,NaN
4,02d48cf1-1ae6-40b3-9d8b-8208884a0904,A,Saturday


## Calculating Purchase Rates

We need to help Brian determine whether or not there is a significant difference in the percent of users who purchased the upgrade package among groups A, B, and C.

***

1. Define a new column called `is_purchase` which is `"Purchase"` if `click_day` is not `None` and `"No Purchase"` if `click_day` is `None`. This will tell us if each visitor clicked on the purchase link.

In [4]:
df['is_purchase'] = df.click_day.apply(lambda x: "Purchase" if pd.notnull(x) else "No Purchase")

df.head()

,user_id,group,click_day,is_purchase
0,8e27bf9a-5b6e-41ed-801a-a59979c0ca98,A,NaN,No Purchase
1,eb89e6f0-e682-4f79-99b1-161cc1c096f1,A,NaN,No Purchase
2,7119106a-7a95-417b-8c4c-092c12ee5ef7,A,NaN,No Purchase
3,e53781ff-ff7a-4fcd-af1a-adba02b2b954,A,NaN,No Purchase
4,02d48cf1-1ae6-40b3-9d8b-8208884a0904,A,Saturday,Purchase


2. We want to count the number of users who made a purchase from each group. Use `groupby` to count the number of `"Purchase"` and `"No Purchase"` from each `group`. Save your answer to the variable `purchase_counts`.

In [5]:
purchase_counts = df.groupby(['group', 'is_purchase']).user_id.count().reset_index()

purchase_counts

,group,is_purchase,user_id
0,A,No Purchase,1350
1,A,Purchase,316
2,B,No Purchase,1483
3,B,Purchase,183
4,C,No Purchase,1583
5,C,Purchase,83


3. Examine `purchase_counts`.

***

## Performing a Significance Test

Here is our table of purchases from the previous exercise:

|group|is_purchase|user_id|
|:----|:----------|:------|
|A|No Purchase|1350|
|A|Purchase|316|
|B|No Purchase|1483|
|B|Purchase|183|
|C|No Purchase|1583|
|C|Purchase|83|

The data from this A/B test is *categorical* data.

Why?

Because a users' response can be either `"Purchase"` or `"No Purchase"`.

There are *more* than 2 conditions: users could be in either Group A, Group B, or Group C.

Recall our table for determining which significance test to use:

|  |Numerical|Categorical|
|:-|:-------|:----------|
|**Sample vs. Known Quantity**|1 Sample T-Test|Binomial Test|
|**2 Samples**|2 Sample T-Test|Chi Square|
|**More Than 2 Samples**|ANOVA and/or Tukey|Chi Square|

Which type of test should we use?

***

1. You should perform a chi-squared test to determine if the differences between Groups A, B, and C are significant.

    Import `chi2_contingency` from `scipy.stats` so that you can perform the chi-squared test.

In [6]:
from scipy.stats import chi2_contingency

2. The function `chi2_contingency` accepts one argument: a contingency table.

    Start by filling in the contingency table below with the correct values:

        contingency = [[A_purchases, A_not_purchases],
                       [B_purchases, B_not_purchases],
                       [C_purchases, C_not_purchases]]

In [7]:
contingency_df = df.pivot_table(
    columns='is_purchase',
    index='group',
    values='user_id',
    aggfunc='count').reset_index()

contingency_df

is_purchase,group,No Purchase,Purchase
0,A,1350,316
1,B,1483,183
2,C,1583,83


In [8]:
contingency = []
for _, row in contingency_df.iterrows():
    contingency.append([row['Purchase'], row['No Purchase']])

for item in contingency:
    print(item)

[316, 1350]
[183, 1483]
[83, 1583]


3. Use the function `chi2_contingency` with the data in `contingency` to calculate the p-value.

    Save only the p-value to the variable `pvalue`.

In [9]:
_, pvalue, _, _ = chi2_contingency(contingency)

pvalue

2.4126213546684264e-35

4. Is `pvalue` less than $0.05$? If so, there is a significant difference between the three groups.

    Create a variable called `is_significant` and make it:

* `True` if pvalue is less than $0.05$
* `False` if pvalue is greater than $0.05$

In [10]:
significance_threshold = 0.05

is_significant = pvalue < significance_threshold

is_significant

True

***

## Calculating Necessary Purchase Rates

Your day is a little less busy than you expected, so you decide to ask Brian about his test.

>**You**: Hey Brian! What was that test you were running anyway?

>**Brian**: It was awesome! We are trying to get users to purchase a small FarmBurg upgrade package. It is called a microtransaction. We are not sure how much to charge for it, so we tested three different price points: \$0.99, \$1.99, and \$4.99. It looks like significantly more people bought the upgrade package for \$0.99, so I guess that is what we will charge.

>**You**: Oh no! I should have asked you this before we did that chi-squared test. I do not think that this was the right test at all. It is true that more people wanted to purchase the upgrade at \$0.99; you probably expected that. What we really want to know is if each price point allows us to make enough money that we can exceed some target goal. Brian, how much do you think it cost to build this feature?

>**Brian**: Hmm. I guess that we need to generate a minimum of \$1000 per week in order to justify this project.

>**You**: We have some work to do!

***

1. How many visitors came to the site this week? Save your answer to `num_visits`.

In [11]:
num_visits = len(df)

2. Let us assume that `num_visits` is how many visitors we generally get each week. Given that, calculate the percent of visitors who would need to purchase the upgrade package at each price point (\$0.99, \$1.99, \$4.99) in order to generate Brian's target of \$1,000 per week.

    Save the results to:

    * `p_clicks_099`
    * `p_clicks_199`
    * `p_clicks_499`

    Note that for higher price points, you will need to sell fewer upgrade packages in order to meet your target.

In [12]:
sales_target = 1000

p_clicks_099 = sales_target / (0.99 * num_visits)
p_clicks_199 = sales_target / (1.99 * num_visits)
p_clicks_499 = sales_target / (4.99 * num_visits)

print(f"Increase required for 0.99 microtransaction: {p_clicks_099:.2%}")
print(f"Increase required for 1.99 microtransaction: {p_clicks_199:.2%}")
print(f"Increase required for 4.99 microtransaction: {p_clicks_499:.2%}")

Increase required for 0.99 microtransaction: 20.21%
Increase required for 1.99 microtransaction: 10.05%
Increase required for 4.99 microtransaction: 4.01%


***

Performing a Significance Test II

Here is our table of purchases again:

|group|is_purchase|user_id|
|:----|:----------|:------|
|A|No Purchase|1350|
|A|Purchase|316|
|B|No Purchase|1483|
|B|Purchase|183|
|C|No Purchase|1583|
|C|Purchase|83|

We want to see if the percent of Group A that purchased an upgrade package is significantly greater than `p_clicks_099` (the percent of visitors who need to buy an upgrade package at \$0.99 in order to make our target of \$1,000).

We are comparing a single set of samples to a target. Our data is still categorical.

Recall our table for determining which significance test to use:

|  |Numerical|Categorical|
|:-|:-------|:----------|
|**Sample vs. Known Quantity**|1 Sample T-Test|Binomial Test|
|**2 Samples**|2 Sample T-Test|Chi Square|
|**More Than 2 Samples**|ANOVA and/or Tukey|Chi Square|

Which type of test should we use?

***

1. We should use a **binomial test** on each group to see if the observed purchase rate is significantly greater than what we need in order to generate at least $1,000 per week.

    Import `binom_test` from `scipy.stats`.

In [13]:
from scipy.stats import binom_test

2. For Group A (\$0.99 price point), perform a `binom_test` to see if the observed purchase rate is significantly greater than `p_clicks_099`.

    * `x` will be the number of purchases for Group A
    * `n` will be the total number of visitors assigned Group A
    * `p` will be the target percent of purchases for that price point (calculated above as `p_clicks_099`)
    
    Save the results to `pvalueA`.

In [14]:
x = int(contingency_df['Purchase'][contingency_df['group'] == 'A'])
n = int(contingency_df['Purchase'][contingency_df['group'] == 'A']
    +   contingency_df['No Purchase'][contingency_df['group'] == 'A'])
p = p_clicks_099

pvalueA = binom_test(x, n=n, p=p)

pvalueA

0.2111287299402726

3. For Group B (\$1.99 price point), perform a binom_test to see if the observed purchase rate is significantly greater than `p_clicks_199`.

    Save the results to `pvalueB`.

In [15]:
x = int(contingency_df['Purchase'][contingency_df['group'] == 'B'])
n = int(contingency_df['Purchase'][contingency_df['group'] == 'B']
    +   contingency_df['No Purchase'][contingency_df['group'] == 'B'])
p = p_clicks_199

pvalueB = binom_test(x, n=n, p=p)

pvalueB

0.20660209246555486

4. For Group C (\$4.99 price point), perform a `binom_test` to see if the observed purchase rate is significantly greater than `p_clicks_499`.

    Save the results to `pvalueC`.

In [16]:
x = int(contingency_df['Purchase'][contingency_df['group'] == 'C'])
n = int(contingency_df['Purchase'][contingency_df['group'] == 'C']
    +   contingency_df['No Purchase'][contingency_df['group'] == 'C'])
p = p_clicks_499

pvalueC = binom_test(x, n=n, p=p)

pvalueC

0.045623672477172125

5. What price should Brian charge for the upgrade package? Save your answer to the variable `final_answer`.

In [17]:
final_answer = 4.99